In [ ]:
import sys
np.set_printoptions(threshold=sys.maxsize)

In [38]:
#need location of image that is the brightest

import time
import cv2
import numpy as np
from realsense_depth import *
from IPython.display import clear_output

buffer = [0] * 40  #buffer moving average filter
dc = DepthCamera() #initialize intel realsense

#set frames (if no red object detected)
#depth frame 2 is colorized - for viewing only
ret, depth_frame, color_frame, depth_frame2 = dc.get_frame()
rows, cols, _ = color_frame.shape
x_medium = int(cols/2)
y_medium = int(rows/2)
center = int(cols/2)


while True:
    
    #stream from intel realsense
    ret, depth_frame, color_frame, heat_frame = dc.get_frame()
    depth_frameDISP = cv2.normalize(depth_frame, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    depth_frame = depth_frame[:,100:]
    color_frame = color_frame[:,100:]
    
    #smoothing kernel
    n = 50
    kernel = np.ones((n,n), np.uint8)/n
    
    #get mask for all pixels within distance threshold
    distMask = np.where(depth_frame < 800, depth_frame, 0)
    distMask = distMask.astype(np.uint8)
    
    #get mask for all pixels exceeding the distance threshold
    distPass = np.where(depth_frame >= 800, depth_frame, 0)
    distPass = cv2.filter2D(src=distPass, ddepth=-1, kernel=kernel)
    
    #open area
    distPass = np.where(distPass == np.amax(distPass), distPass, 0)
    com = np.where(distPass == np.amax(distPass))
    mX = np.mean(com[1])
    mY = np.mean(com[0])
    cv2.circle(color_frame, (int(mX), int(mY)), 10, (0, 0, 255), -1)
    
    #finding largest object
    contours, _ = cv2.findContours(distMask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key=lambda x:cv2.contourArea(x), reverse=True)

    #this follows the largest object within the distance
    for cnt in contours:
        (x, y, w, h) = cv2.boundingRect(cnt)
        cv2.rectangle(distMask, (x,y), (x + w, y + h), (0, 255, 0), 2)
        cv2.rectangle(color_frame, (x,y), (x + w, y + h), (0, 255, 0), 2)
        x_medium = int((x + x + w)/2)
        y_medium = int((y + y + h)/2)
        cv2.circle(color_frame, (x_medium,y_medium), 5, (0, 255, 0))
        break
    
    #moving avg buffer for stable output distance
    dist1 = depth_frame[y_medium, x_medium]
    buffer[0] = dist1
    for i in range(len(buffer)):
        buffer[len(buffer)-1-i] = buffer[len(buffer)-2-i]
    dist1 = round(np.mean(dist1)/100)*100
    
    if mX < x_medium:
        cv2.circle(color_frame, (20 ,int(rows/2)), 20, (0, 255, 0), -1)
    else:
        cv2.circle(color_frame, (len(color_frame[1])-20,int(rows/2)), 20, (0, 255, 0), -1)
    
    clear_output(wait=True)
    print(dist1)
    
    #shows frame
    cv2.imshow('filt', distPass)
    cv2.imshow('distMask', distMask)
    cv2.imshow('color_frame', color_frame)
    key = cv2.waitKey(1)
    if key == 27:
        break
        
cv2.destroyAllWindows

600


KeyboardInterrupt: 